<a href="https://colab.research.google.com/github/konerjonlar/Akbank-Makine-Ogrenmesi-Bootcamp/blob/main/Basit_Dogrusal_Resgresyon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
df = pd.read_csv("insurance.csv")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [7]:
df.isna().sum().sum()

0

In [10]:
numerik_sutunlar = df.select_dtypes(include=['number'])

for sutun in numerik_sutunlar.columns:
    Q1 = df[sutun].quantile(0.25)
    Q3 = df[sutun].quantile(0.75)
    IQR = Q3 - Q1

    alt_sinir = Q1 - 1.5 * IQR
    ust_sinir = Q3 + 1.5 * IQR

    df = df[(df[sutun] >= alt_sinir) & (df[sutun] <= ust_sinir)]

In [12]:
def veri_tipleri_belirle(veri_cercevesi, cat_esik=10, car_esik=20):
    """
    Veri çerçevesindeki sütunları kategorik ve sayısal olarak ayırmak için bir işlev.

    :param veri_cercevesi: Veri çerçevesi
    :param cat_esik: Kategorik olarak kabul edilen sütun için eşik değer
    :param car_esik: Sayısal olarak kabul edilen sütun için eşik değer
    """
    # Kategorik sütunları belirle
    kategorik_sutunlar = [sutun for sutun in veri_cercevesi.columns if veri_cercevesi[sutun].dtypes == "O"]

    # Sayısal ama kategorik davranan sütunları belirle
    sayisal_ama_kategorik = [sutun for sutun in veri_cercevesi.columns if veri_cercevesi[sutun].nunique() < cat_esik and veri_cercevesi[sutun].dtypes != "O"]

    # Kategorik ama sayısal davranan sütunları belirle
    kategorik_ama_sayisal = [sutun for sutun in veri_cercevesi.columns if veri_cercevesi[sutun].nunique() > car_esik and veri_cercevesi[sutun].dtypes == "O"]

    # Kategorik sütunlar listesini güncelle
    kategorik_sutunlar = kategorik_sutunlar + sayisal_ama_kategorik

    # Sayısal sütunlar listesinden kategorik ama sayısal davranan sütunları çıkar
    kategorik_sutunlar = [sutun for sutun in kategorik_sutunlar if sutun not in kategorik_ama_sayisal]

    # Sayısal sütunları belirle
    sayisal_sutunlar = [sutun for sutun in veri_cercevesi.columns if veri_cercevesi[sutun].dtypes != "O"]

    # Sayısal sütunlar listesinden sayısal ama kategorik davranan sütunları çıkar
    sayisal_sutunlar = [sutun for sutun in sayisal_sutunlar if sutun not in sayisal_ama_kategorik]

    print(f"Veri Sayısı: {veri_cercevesi.shape[0]}")
    print(f"Değişken Sayısı: {veri_cercevesi.shape[1]}")
    print(f"Kategorik Değişkenler: {len(kategorik_sutunlar)}")
    print(f"Sayısal Değişkenler: {len(sayisal_sutunlar)}")
    print(f"Kategorik Değişkenler (Sayısal Gibi Davrananlar): {len(sayisal_ama_kategorik)}")
    print(f"Sayısal Değişkenler (Kategorik Gibi Davrananlar): {len(kategorik_ama_sayisal)}")
veri_tipleri_belirle(df)

Veri Sayısı: 1191
Değişken Sayısı: 7
Kategorik Değişkenler: 4
Sayısal Değişkenler: 3
Kategorik Değişkenler (Sayısal Gibi Davrananlar): 1
Sayısal Değişkenler (Kategorik Gibi Davrananlar): 0


In [15]:
# Orijinal veriyi değiştirmemek için veri çerçevesinin bir kopyasını oluşturun
df_encoded = df.copy()

# 'sex' ve 'smoker' sütunlarına Label Encoding uygula
label_encoder = LabelEncoder()
df_encoded['sex'] = label_encoder.fit_transform(df_encoded['sex'])
df_encoded['smoker'] = label_encoder.fit_transform(df_encoded['smoker'])

# 'region' sütununa One-Hot Encoding uygula

# One-Hot Encoding işlemine tabi tutulan 'region' sütununu seç
region_sutunu = df_encoded[['region']]

# One-Hot Encoder nesnesini oluştur
onehot_encoder = OneHotEncoder(drop='first', sparse=False)

# One-Hot Encoding işlemi uygula
encoded_region = onehot_encoder.fit_transform(region_sutunu)

# Kodlanmış 'region' sütununu bir veri çerçevesine dönüştür
encoded_region_df = pd.DataFrame(encoded_region, columns=onehot_encoder.get_feature_names_out(['region']))

# Kodlanmış 'region' sütununu veri çerçevesi ile birleştir
df_encoded = pd.concat([df_encoded, encoded_region_df], axis=1)

# Artık işe yaramayan 'region' sütununu kaldır
df_encoded.drop(['region'], axis=1, inplace=True)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [17]:
# Bağımsız değişkenleri (X) ve bağımlı değişkeni (y) ayır
X = df_encoded.drop(['charges'], axis=1)  # 'charges' sütununu bağımlı değişken olarak ayır
y = df_encoded['charges']  # 'charges' sütununu bağımlı değişken olarak atama

# Satırları düşürdükten sonra indeksleri sıfırla
X.reset_index(drop=True, inplace=True)  # X verisinin indekslerini sıfırla
y.reset_index(drop=True, inplace=True)  # y verisinin indekslerini sıfırla

# NaN (boş) değer içeren satırları düşür
X.dropna(inplace=True)  # X verisindeki NaN değerlere sahip satırları kaldır
y = y[X.index]  # y verisini uygun indekslere göre güncelle


In [18]:
# Veriyi eğitim ve test setlerine böl
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Veriyi Standard Scaling kullanarak ölçeklendir
olceklendirici = StandardScaler()
X_egitim_olceklendirilmis = olceklendirici.fit_transform(X_egitim)
X_test_olceklendirilmis = olceklendirici.transform(X_test)


In [19]:
# LinearRegression modelini oluştur
model = LinearRegression()

# Modeli eğit
model.fit(X_egitim_olceklendirilmis, y_egitim)

LinearRegression()

In [22]:
# Modelin tahminlerini yap
tahminler = model.predict(X_test_olceklendirilmis)

# Ortalama Kare Hata (Mean Squared Error, MSE) hesapla
mse = mean_squared_error(y_test, tahminler)
print(f"Ortalama Kare Hata (MSE): {mse}")

# Ortalama Mutlak Hata (Mean Absolute Error, MAE) hesapla
mae = mean_absolute_error(y_test, tahminler)
print(f"Ortalama Mutlak Hata (MAE): {mae}")

# R-Kare (R-Squared) hesapla
r2 = r2_score(y_test, tahminler)
print(f"R-Kare (R-Squared) Değeri: {r2}")


Ortalama Kare Hata (MSE): 22929505.64965124
Ortalama Mutlak Hata (MAE): 2726.917750777625
R-Kare (R-Squared) Değeri: 0.5638966368231884
